# Machine Learning Assignment – Naive Bayes, Decision Trees & Ensemble Learning

This notebook contains implementations and answers to conceptual questions on Naive Bayes, Decision Trees, and Ensemble Learning.

## Part I: Naive Bayes Classifier

### Task 1: Theory Questions
1. **What is the core assumption of Naive Bayes?**  
   It assumes that all features are conditionally independent given the class label.

2. **Differentiate between GaussianNB, MultinomialNB, and BernoulliNB.**  
   - GaussianNB: For continuous features assuming Gaussian distribution.  
   - MultinomialNB: For count features (e.g., word counts).  
   - BernoulliNB: For binary features.

3. **Why is Naive Bayes considered suitable for high-dimensional data?**  
   It's computationally efficient and handles large feature spaces well due to independence assumption.

### Task 2: Spam Detection using MultinomialNB

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Load SMS Spam Collection Dataset (replace this path if needed)
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

# Encode labels
df['label_num'] = df.label.map({'ham': 0, 'spam': 1})

# Vectorize
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['message'])
y = df['label_num']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


### Task 3: GaussianNB with Iris Dataset

In [ ]:

from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)
print("GaussianNB Classification Report:\n", classification_report(y_test, y_pred_gnb))

# Compare with Logistic Regression
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_lr))

# Compare with Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))


## Part II: Decision Trees


### Task 4: Conceptual Questions

1. **What is entropy and information gain?**  
   - Entropy measures the impurity or uncertainty in a dataset.  
   - Information gain is the reduction in entropy after a dataset is split on an attribute.

2. **Explain the difference between Gini Index and Entropy.**  
   - Gini measures impurity by calculating the probability of misclassification.  
   - Entropy uses logarithmic calculations to quantify impurity.

3. **How can a decision tree overfit? How can this be avoided?**  
   - A tree can overfit by growing too deep and capturing noise.  
   - Use pruning, max depth limits, and minimum samples per split to avoid overfitting.


### Task 5: Decision Tree on Titanic Dataset

In [ ]:

import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder

# Load Titanic dataset
df = sns.load_dataset('titanic')
df = df[['survived', 'pclass', 'sex', 'age', 'fare', 'embarked']].dropna()

# Encode categorical
df['sex'] = LabelEncoder().fit_transform(df['sex'])
df['embarked'] = LabelEncoder().fit_transform(df['embarked'])

# Split
X = df.drop('survived', axis=1)
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Evaluate
y_pred = tree_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Plot
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 8))
plot_tree(tree_model, feature_names=X.columns, class_names=["Not Survived", "Survived"], filled=True)
plt.show()


### Task 6: Model Tuning

In [ ]:

train_acc = []
test_acc = []
depths = range(1, 15)

for d in depths:
    model = DecisionTreeClassifier(max_depth=d, random_state=42)
    model.fit(X_train, y_train)
    train_acc.append(model.score(X_train, y_train))
    test_acc.append(model.score(X_test, y_test))

plt.plot(depths, train_acc, label="Train Accuracy")
plt.plot(depths, test_acc, label="Test Accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.title("Training vs Testing Accuracy")
plt.legend()
plt.grid(True)
plt.show()


## Part III: Ensemble Learning – Bagging, Boosting, Random Forest


### Task 7: Conceptual Questions

1. **What is the difference between Bagging and Boosting?**  
   - Bagging trains multiple models independently in parallel.  
   - Boosting trains models sequentially, each learning from previous errors.

2. **How does Random Forest reduce variance?**  
   - By averaging predictions of multiple decision trees trained on different data subsets.

3. **What is the weakness of boosting-based methods?**  
   - They are sensitive to noisy data and outliers.


### Task 8: Random Forest vs Decision Tree

In [ ]:

from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Precision:", precision_score(y_test, y_pred_rf))
print("Random Forest Recall:", recall_score(y_test, y_pred_rf))

# Feature importances
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', title="Feature Importances")
plt.show()


### Task 9: AdaBoost or Gradient Boosting

In [ ]:

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score
import time

# AdaBoost
start = time.time()
ab = AdaBoostClassifier(n_estimators=100, random_state=42)
ab.fit(X_train, y_train)
y_pred_ab = ab.predict(X_test)
end = time.time()

print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ab))
print("AdaBoost F1 Score:", f1_score(y_test, y_pred_ab))
print("Training Time:", round(end - start, 2), "seconds")

# Gradient Boosting
start = time.time()
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
end = time.time()

print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Gradient Boosting F1 Score:", f1_score(y_test, y_pred_gb))
print("Training Time:", round(end - start, 2), "seconds")
